In [5]:
import dataiku
import dataikuapi
from dataiku import pandasutils as pdu
import pandas as pd

In [3]:
design_host = dataiku.api_client().host 
print(design_host)

http://127.0.0.1:20001


In [6]:
design_client = dataikuapi.DSSClient(design_host, "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [7]:
project = design_client.get_project("DSPIPELINEFRAUD")

In [11]:
project_permissions = project.get_permissions().get("permissions")

In [15]:
dss_users = design_client.list_users()

user_list = []
# Grab list of users where they have active web socket sessions
for user in dss_users:
    if user['activeWebSocketSesssions'] != 0:
        user_list.append(user['displayName'])
print(user_list)

['Administrator']


In [16]:
dss_users

[{'activeWebSocketSesssions': 2,
  'atSurveySettings': {'nextAction': 1653828056730,
   'state': 'INITIAL',
   'surveys': {'ChartsOpen': {'nextAction': 1652960118883},
    'DashboardEdit': {'nextAction': 1653828056730}}},
  'displayName': 'Administrator',
  'enabled': True,
  'groups': ['administrators'],
  'login': 'admin',
  'npsSurveySettings': {'nextAction': 1658925994517, 'state': 'RECURRING'},
  'objectImgHash': 0,
  'sourceType': 'LOCAL',
  'userProfile': 'DATA_SCIENTIST'},
 {'activeWebSocketSesssions': 0,
  'atSurveySettings': {'nextAction': 0, 'state': 'INITIAL', 'surveys': {}},
  'displayName': 'Sample business user',
  'enabled': True,
  'groups': ['readers'],
  'login': 'viewer',
  'npsSurveySettings': {'nextAction': 1641764273123, 'state': 'INITIAL'},
  'objectImgHash': 0,
  'sourceType': 'LOCAL',
  'userProfile': 'DATA_SCIENTIST'},
 {'activeWebSocketSesssions': 0,
  'atSurveySettings': {'nextAction': 0, 'state': 'INITIAL', 'surveys': {}},
  'displayName': 'user0',
  'enab

In [21]:
bundle_id = project.project_key
print(bundle_id)

DSPIPELINEFRAUD


In [26]:
#bundle_version_type = config.get("bundle_version_type", "")
bundle_version_type = "minor"

In [44]:
bundle_id + '_v' + str(version_id)

'DSPIPELINEFRAUD_v1'

In [45]:
exported_bundles = ["DSPIPELINEFRAUD_v1", "DSPIPELINEFRAUD_v1.1", "DSPIPELINEFRAUD_v1.2", "DSPIPELINEFRAUD_v2"]

In [46]:
version_id = 1
#exported_bundles = project.list_exported_bundles()['bundles']
if len(exported_bundles) == 0: 
    if bundle_version_type == "major":
        version_id = 1
    elif bundle_version_type == "minor":
        raise Exception("This is the first time you deploy the project, it should be a major version!") 
elif len(exported_bundles) > 0:
    last_deployed_bundles = exported_bundles[-1]
    

In [47]:
last_deployed_bundles.split('_')

'DSPIPELINEFRAUD_v2'

In [17]:
project.list_exported_bundles()

{'bundles': []}

In [18]:
project.list_imported_bundles()

{'bundles': []}